In [1]:
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [5]:
import os
import os.path as osp
import numpy as np

In [54]:
# def read_y(vot_path):
vot_path = "/home/zabulskyy/Datasets/vot2016/"
if 1:
    vot_classes = [x for x in os.listdir(vot_path) if not x.endswith(".txt")]
    y = np.array([])
    for vot_class in vot_classes:
        gt_file = osp.join(vot_path, vot_class, "groundtruth.txt")
        with open(gt_file, 'r') as file:
            l =file.read().split("\n")
            l.pop(-1)
            l = [[float(x) for x in y.split(",")] for y in l]
            l = [[min(m[::2]), min(m[1::2]), max(m[::2]), max(m[1::2])] for m in l]
            y = np.append(y, np.array(l))
